Oilco must determine how many barrels of oil to extract during each of the next 2 years. If Oilco extracts $x_1$ million barrels during Year 1, then each barrel can be sold for $30-x_1$ dollars. If Oilco extracts $x_2$ million barrels during Year 2, then each barrel can be sold for $35-x_2$ dollars. The cost of extracting $x_1$ barrels in Year 1 is $x_1^2$ million dollars and the cost of extracting $x_2$ barrels in Year 2 is $2x_2^2$ million dollars. A total of 20 million barrels are available and a max of $250 million can be spent on extraction. Formulate a MINLP problem to help Oilco maximize profits (revenue - costs) for the next 2 years.

Profit = Revenue-Cost

<center>

$\max \left[x_1(30-x_1)+x_2(35-x_2)-(x_1^2+2x_2^2)\right]=$

$\max \left[30x_1-x_1^2+35x_2-x_2^2-x_1^2-2x_2^2\right]=$

$\max \left[-2x_1^2-3x_2^2+30x_1+35x_2\right]$</center>

subject to:


<center>

$x_1+x_2 \le 20$

$x_1^2+2x_2^2\le250$

$x_1,x_2$ are integers</center>


In [9]:
# Ipopt installer

import sys

if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py"
    import colab_helper
    colab_helper.install_idaes()
    colab_helper.install_ipopt()

--2025-08-21 18:06:16--  https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5459 (5.3K) [text/plain]
Saving to: ‘colab_helper.py.1’

colab_helper.py.1   100%[===================>]   5.33K  --.-KB/s    in 0s      

2025-08-21 18:06:16 (55.8 MB/s) - ‘colab_helper.py.1’ saved [5459/5459]

idaes was found! No need to install.


In [10]:
import pyomo.environ as pyo

We'll start with linear programming problem to be solved by a non-linear optimizer, `ipopt`.

In [23]:
model = pyo.ConcreteModel()

model.x1 = pyo.Var(bounds=(0, None), within=pyo.Integers)
model.x2 = pyo.Var(bounds=(0, None),within=pyo.Integers)

x1 = model.x1
x2 = model.x2

model.C1 = pyo.Constraint(expr=x1 + x2 <= 20)
model.C2 = pyo.Constraint(expr=x1**2 + 2*(x2**2) <= 250)


model.obj = pyo.Objective(expr=-2*(x1**2)-3*(x2**2)+30*x1+35*x2, sense=pyo.maximize)

In [24]:
opt = pyo.SolverFactory('mindtpy')
opt.solve(model, mip_solver='cbc')

{'Problem': [{'Name': 'unknown', 'Lower bound': 214.0000068524824, 'Upper bound': 214.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of binary variables': 0, 'Number of integer variables': 2, 'Number of continuous variables': 0, 'Number of nonzeros': None, 'Sense': 'maximize', 'Number of disjunctions': 0}], 'Solver': [{'Name': 'MindtPyOA', 'Status': 'ok', 'Message': None, 'User time': 0.21395240200035914, 'Wallclock time': 0.21395240200035914, 'Termination condition': 'optimal', 'Termination message': None, 'Timing': Bunch(Call after main solve = 2.4934000066423323e-05, Call after subproblem solve = 1.3009000213060062e-05, Call before subproblem solve = 1.0086000202136347e-05, OA cut generation = 0.0026232479995087488, fixed subproblem = 0.06324624499939091, initialization = 0.02199740500009284, main = 0.1025815899997724, main loop = 0.18017822699994213, main_timer_start_time = 2341.350318428, total = 0.21395240200035914), 'Iterations': 5, '

In [25]:
model.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   8.0 :  None : False : False : Integers
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   6.0 :  None : False : False : Integers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : -2*x1**2 - 3*x2**2 + 30*x1 + 35*x2

2 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : x1 + x2 :  20.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :  -Inf : x1**2 + 2*x2**2 : 250.0 :   True

5 Declarations: x1 x2 C1 C2 obj


In [26]:

x1_value = pyo.value(x1)
x2_value = pyo.value(x2)
print("x1 =", x1_value)
print("x2 =", x2_value)
print("Profit =", model.obj())

x1 = 8.0
x2 = 6.0
Profit = 214.0
